In [1]:
from utils import *
import pandas
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
# optimization
from tensorflow.keras import mixed_precision
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

policy = tf.keras.mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [3]:
df = pandas.read_excel("dictionary.xlsx")
df = np.array(df)[1:]
print(df.shape)
data_folder = "E2E_dataset"
eyes_labelled = load_eye_tensors_with_labels(data_folder, df)

eyes, labels = filter_and_normalize(eyes_labelled)

(6757, 3)
Loaded ACEL01E.E2E
Loaded ACEL02E.E2E
Loaded ACEL03E.E2E
Loaded ACEL04E.E2E
Loaded ACEL05E.E2E
Loaded ACEL11E.E2E
Loaded ACEL13E.E2E
Loaded ACEL14E.E2E
Loaded ACEL15E.E2E
Loaded ACEL19E.E2E
Loaded ACEL45E.E2E
Loaded ACEL46E.E2E
Loaded ACEL47E.E2E
Loaded ADAM01K.E2E
Loaded ADAM02K.E2E
Loaded ADAM03K.E2E
Loaded ADAM04K.E2E
Loaded ADAMI03P.E2E
Loaded AGOTA01Z.E2E
Loaded AGOTA02Z.E2E
Loaded AGYAG10I.E2E
Loaded AJTAY01M.E2E
Loaded ALMAS01I.E2E
Loaded ALMAS02I.E2E
Loaded ALMAS03I.E2E
Loaded ALMAS04I.E2E
Loaded ALMAS05I.E2E
Loaded ALMAS06I.E2E
Loaded ALMAS07I.E2E
Loaded ALMAS08I.E2E
Loaded ALMAS09I.E2E
Loaded ALMAS10I.E2E
Loaded ALMAS11I.E2E
Loaded ALMAS12I.E2E
Loaded ALMAS13I.E2E
Loaded ALMAS14I.E2E
Loaded ALMAS15I.E2E
File ALMAS15I.E2E issue index 0 is out of bounds for axis 0 with size 0
Loaded ALMAS16I.E2E
Loaded ALMAS17I.E2E
File ALMAS17I.E2E issue index 0 is out of bounds for axis 0 with size 0
Loaded ALMAS18I.E2E
Loaded ALMAS19I.E2E
Loaded ALMAS20I.E2E
Loaded ALMAS21I.E2E
Loa

In [16]:
unique = set()
for eye in eyes:
    if eye.shape not in unique:
        unique.add(eye.shape)
print(unique)
eyes = np.expand_dims(eyes, axis=-1)
print(eyes.shape)
labels = np.expand_dims(labels, axis=-1)
print(labels.shape)

{(25, 496, 512, 1)}
(71, 25, 496, 512, 1, 1)
(71, 1)


In [18]:
input_img = tf.keras.Input(shape=(25, 496, 512, 1))
x = layers.Conv3D(16, (3, 3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling3D((1, 2, 2), padding='same')(x)
x = layers.Conv3D(8, (3, 3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling3D((1, 2, 2), padding='same')(x)

x = layers.Conv3D(8, (3, 3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling3D((1, 2, 2))(x)  # Output: (25, 248, 256, 8)
x = layers.Conv3D(16, (3, 3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling3D((1, 2, 2))(x)
decoded = layers.Conv3D(1, (3, 3, 3), activation='sigmoid', padding='same')(x)

In [19]:
autoencoder = models.Model(input_img, decoded)
autoencoder.summary()
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((eyes, eyes)).batch(1).shuffle(buffer_size=71)

In [ ]:
autoencoder.fit(dataset, epochs=20)

Epoch 1/10
1/9 [==>...........................] - ETA: 14:40 - loss: 0.6933

In [ ]:
encoder = models.Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(index=4).output)  # Use last encoded layer
for layer in encoder.layers:
    layer.trainable = False

input_img = tf.keras.Input(shape=(25, 496, 512, 1))
x = encoder(input_img)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(1, activation='sigmoid')(x)

In [ ]:
model = models.Model(input_img, x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(eyes, labels, epochs=10, batch_size=8, shuffle=True)